# 1- Webscaping for Toronto neighborhoods information

In [14]:
import pandas as pd
import bs4 as bs
!pip install folium
import folium
print ('Import phase completed')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 7.0 MB/s  eta 0:00:01
Import phase completed


### I'm using beautifulsoup to extract data from the wikipedia page. Then in the cycle I'm getting off all the "Not Assigned" Borough. Finally I manage multiple neighborhoods within the same borough by adding all of them in the same cells and separating them with commas.

In [2]:
## Sample code
import urllib.request
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
sauce=urllib.request.urlopen(url).read()
soup= bs.BeautifulSoup(sauce,'lxml')
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Finally I use the shape function to understand dataframe dimensions

In [3]:
print(df.shape)

(103, 3)


# 2- Downloading latitude and longitude dataframe and merge the two dfs 

In [4]:
import pandas as pd
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
geo_loc_df=pd.read_csv(url)
geo_loc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
tor_df = pd.merge(left=df, right=geo_loc_df, left_on='PostalCode', right_on='Postal Code') ##Merging
tor_df.drop(labels='Postal Code', axis=1, inplace=True) ##Eliminating the duplicate column
tor_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### 3 Let's now create a map of Toronto Neighborhoods 

#### We will use the folium package. To build the map the following code will take the table we created in previous cells and it's going to need starting point for Latitude and Longitude. We will considere the Average values for Latitude and Longitude computed among all neighborhoods.

In [20]:
#Computing initial latitude and longitude
In_Latitude=tor_df['Latitude'].mean()
In_Longitude=tor_df['Longitude'].mean()

# create map of Toronto using initial latitude and longitude values that we've just computed
map_toronto = folium.Map(location=[In_Latitude, In_Longitude], zoom_start=11)

#Cycle to add a circle for each neighborhood
for lat, lng, borough, neighborhood in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Borough'], tor_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto